In [3]:
# importer packages
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import sqlite3
import time
import requests
from bs4 import BeautifulSoup
import re

In [4]:
# Instantier l'objet chrome options
options = webdriver.ChromeOptions()
# définir l'option d'utiliser chrome en mode headless ( utiliser afin de lancer le script en background)
options.add_argument("--headless=new")
# initialiser l'instance de chrome driver en mode headless
driver = webdriver.Chrome(options=options)

Fonction de nettoyage des textes

In [5]:
def clean_text(text):
    return re.sub(r'\s+', ' ', text.strip().replace('\u202f', ''))

In [12]:
data = []

for page in range(1, 51):
    url = f"https://dakar-auto.com/senegal/voitures-4?page={page}"
    print(f"🔎 Scraping page {page}...")

    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        annonces = soup.select(".listings-cards__list-item")

        for annonce in annonces:
            try:
                etat = clean_text(annonce.select_one("h4 small").text)
                prix = clean_text(annonce.select_one("h3").text).replace("F CFA", "").replace(" ", "")

                # Titre = ex: "Toyota Rav4 2016"
                titre = clean_text(annonce.select_one("h2 a").text)
                parts = titre.split()
                marque = parts[0] if len(parts) > 0 else ""
                modele = " ".join(parts[1:-1]) if len(parts) > 2 else ""
                annee = parts[-1] if parts[-1].isdigit() and len(parts[-1]) == 4 else ""

                # Catégorie
                categorie_tag = annonce.select_one(".listing-card__header__category")
                raw_categorie = clean_text(categorie_tag.text) if categorie_tag else ""
                categorie = raw_categorie.replace(etat, "").strip()

                # Caractéristiques techniques
                km = boite = carburant = ref = ""
                for li in annonce.select("ul.listing-card__attribute-list li"):
                    text = clean_text(li.text)
                    if "km" in text.lower():
                        km = text.replace("km", "").strip()
                    elif "Ref" in text:
                        ref = text.replace("Ref.", "").strip()
                    elif "Manuelle" in text or "Automatique" in text:
                        boite = text
                    elif "Essence" in text or "Diesel" in text:
                        carburant = text

                # Image
                image_tag = annonce.select_one(".listing-card__aside img")
                image_url = "https://dakar-auto.com" + image_tag["src"] if image_tag else ""

                # Adresse (ville + région)
                ville_tag = annonce.select_one(".town-suburb")
                region_tag = annonce.select_one(".province")
                ville = clean_text(ville_tag.text) if ville_tag else ""
                region = clean_text(region_tag.text) if region_tag else ""
                adresse = f"{ville}{region}".strip(", ")

                proprio_tag = annonce.select_one(".time-author")
                proprietaire = clean_text(proprio_tag.text.replace("Par ", "")) if proprio_tag else ""

                data.append({
                    "etat": etat,
                    "prix (FCFA)": prix,
                    "marque": marque,
                    "modele": modele,
                    "annee": annee,
                    "categorie": categorie,
                    "kilometrage (Km)": km,
                    "reference": ref,
                    "boite_vitesse": boite,
                    "carburant": carburant,
                    "adresse": adresse,
                    "proprietaire": proprietaire,
                    "image_url": image_url
                })

            except Exception as e:
                print(f"⚠️ Erreur dans une annonce page {page} : {e}")
                continue

        time.sleep(1)

    except Exception as e:
        print(f"❌ Erreur de chargement de la page {page} : {e}")
        continue

print("✅ Scraping terminé.")

🔎 Scraping page 1...
🔎 Scraping page 2...
🔎 Scraping page 3...
🔎 Scraping page 4...
🔎 Scraping page 5...
🔎 Scraping page 6...
🔎 Scraping page 7...
🔎 Scraping page 8...
🔎 Scraping page 9...
🔎 Scraping page 10...
🔎 Scraping page 11...
🔎 Scraping page 12...
🔎 Scraping page 13...
🔎 Scraping page 14...
🔎 Scraping page 15...
🔎 Scraping page 16...
🔎 Scraping page 17...
🔎 Scraping page 18...
🔎 Scraping page 19...
🔎 Scraping page 20...
🔎 Scraping page 21...
🔎 Scraping page 22...
🔎 Scraping page 23...
🔎 Scraping page 24...
🔎 Scraping page 25...
🔎 Scraping page 26...
🔎 Scraping page 27...
🔎 Scraping page 28...
🔎 Scraping page 29...
🔎 Scraping page 30...
🔎 Scraping page 31...
🔎 Scraping page 32...
🔎 Scraping page 33...
🔎 Scraping page 34...
🔎 Scraping page 35...
🔎 Scraping page 36...
🔎 Scraping page 37...
🔎 Scraping page 38...
🔎 Scraping page 39...
🔎 Scraping page 40...
🔎 Scraping page 41...
🔎 Scraping page 42...
🔎 Scraping page 43...
🔎 Scraping page 44...
🔎 Scraping page 45...
🔎 Scraping page 46.

In [18]:
print(df.columns)


Index(['etat', 'prix (FCFA)', 'marque', 'modele', 'annee', 'categorie',
       'kilometrage (Km)', 'reference', 'boite_vitesse', 'carburant',
       'adresse', 'proprietaire', 'image_url'],
      dtype='object')


In [15]:
df = pd.DataFrame(data)
pd.set_option("display.max_colwidth", None)
df.head(1000)


,etat,prix (FCFA),marque,modele,annee,categorie,kilometrage (Km),reference,boite_vitesse,carburant,adresse,proprietaire,image_url
0,Occasion,5000000,Citroen,C3,2016,Coupés & cabriolets,150000,139356,Manuelle,Diesel,"Thiès,Thiès",Bintou Sylla,https://dakar-auto.com/img/listings/2024/09/IMG-20240927-WA0050_300x194.jpg
1,Occasion,9500000,Mitsubishi,Outlander,2016,4x4s & SUV,115000,139354,Automatique,Essence,"VDN,Dakar",Sokhna Faty,https://dakar-auto.com/img/listings/2024/09/c7788526-02fe-4eae-aee0-0d2bdd276b05_300x194.jpg
2,Occasion,4000000,Renault,megane-3,2013,Berlines & breaks,260000,139352,Manuelle,Diesel,"Sud Foire,Dakar",Gm DK,https://dakar-auto.com/img/listings/2024/09/IMG-20240814-094311-min_300x194.jpg
3,Occasion,15499000,Rover,range,2017,4x4s & SUV,54000,139328,Automatique,Essence,"Médina,Dakar",Chasseur Automobile,https://dakar-auto.com/img/listings/2024/09/Range-rover-evoque-2017_300x194.jpg
4,Occasion,3950000,Ford,Fusion,2013,Berlines & breaks,80000,139330,Automatique,Essence,"Sud Foire,Dakar",Fatou Kiné,https://dakar-auto.com/img/empty_car_logo_300x194.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Occasion,13000000,Ford,Edge,2016,4x4s & SUV,90000,137824,Automatique,Essence,"VDN,Dakar",BAGHDAD AUTOMOBILE,https://dakar-auto.com/img/listings/2022/12/1670405670-1670405670-6_300x194.jpg
996,Occasion,33000000,Toyota,Prado,2015,4x4s & SUV,92000,137825,Automatique,Essence,"Gueule-Tapée, Fass, Colobane,Dakar",Serigne Modou BOUSSOU,https://dakar-auto.com/img/listings/2022/12/1670406109-1670406109-1_300x194.jpg
997,Occasion,4600000,Ford,Focus,2014,Berlines & breaks,184000,137826,Automatique,Essence,"Sicap Sacré Coeur,Dakar",A-T Auto,https://dakar-auto.com/img/listings/2022/12/1670406703-1670406703-2_300x194.jpg
998,Occasion,7900000,Ford,Edge,2015,4x4s & SUV,174563,137827,Automatique,Essence,"Hann Maristes,Dakar",Mr ELA,https://dakar-auto.com/img/listings/2022/12/1670407964-1670407964-1_300x194.jpg


In [17]:
df.to_csv("voitures_clean.csv", index=False, encoding="utf-8-sig")


OSError: Cannot save file into a non-existent directory: 'data\cleaned'